<a href="https://colab.research.google.com/github/levvarvara/NLP/blob/master/2_Language_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://clck.ru/JC6Ea

## Препроцессинг и определение языка / Preprocessing and language detection

Предыдущая часть была посвящена предобработке текстовых данных, теперь применим эти методы на практике и заодно узнаем, как можно решать задачу определения языка.

**Идеи?**

Нам понадобятся некоторые пакеты для Питона, установите себе недостающие:
* wikipedia
* nltk

In [0]:
#!pip install wikipedia

In [0]:
#!pip install nltk

In [0]:
from collections import Counter  # нужно объяснять, что это?

import nltk
import wikipedia

Возможно, перед первым запуском `nltk` нужно скачать немного данных:

In [0]:
#nltk.download()

Теперь посмотрим, как можно скачивать тексты из Википедии с помощью пакета `wikipedia`.

In [0]:
wikipedia.set_lang('ru')
page_name = wikipedia.random(1)
page = wikipedia.page(page_name)
print(page_name)
print(page.title)
print(page.content)

Поскольку мы будем выбирать из нескольких языков, лучше сразу завести функцию, которая будет 
скачивать нужное количество страниц для заданного языка.

In [0]:
TEST_LANGS = ('kk', 'ru', 'uk', 'be', 'fr')

In [0]:
def get_texts_for_lang(lang, n=10):
    wiki_content = []
    ### ВАШ КОД ###
    return wiki_content

In [0]:
wiki_texts = {}
for lang in TEST_LANGS:
    wiki_texts[lang] = get_texts_for_lang(lang, 50)
    print(lang, len(wiki_texts[lang]))

In [0]:
print(wiki_texts['kk'][0])
print(wiki_texts['fr'][0])

**Идея 1**

Можно найти самые частотные слова для каждого языка, а потом считать их количество в каждом новом тексте.

In [0]:
def collect_freqlist(wiki_pages, max_len=100):
    freqlist = Counter()
    ### ВАШ КОД ###
    # не забудем про токенизацию - nltl.word_tokenize
    return dict(freqlist.most_common(max_len))

In [0]:
# проверка
collect_freqlist(wiki_texts['fr'])

In [0]:
freq_lists = {}
for lang in TEST_LANGS:
    freq_lists[lang] = collect_freqlist(wiki_texts[lang])

Теперь всё готово, чтобы сделать первую определялку языка!

In [0]:
def simple_lang_detect(freq_lists, text):
    counts = Counter()
    for lang, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[lang] += int(freq_list[word] > 0)
    return counts.most_common()

In [0]:
test_texts = get_texts_for_lang('fr')[0]
print(test_texts)

In [0]:
simple_lang_detect(freq_lists, test_texts)

Попробуем оценить качество нашей определялки. Для этого установим `sklearn`, если его ещё нет, и возьмем оттуда функцию для подсчёта accuracy.

In [0]:
#!pip install sklearn

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
def test_simple_lang_detect(freq_lists, test_size):
    results = []  # сюда будем писать результаты
    gold = []     # сюда будем писать исходный язык
    for lang in TEST_LANGS:
        ### ВАШ КОД ###
    print("RESULTS:")
    print("%d languages" % len(TEST_LANGS))
    print("Test size: %d texts per language" % test_size)
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [0]:
# проверка
test_simple_lang_detect(freq_lists, 10)

## Машинное обучение

Будем говорить о **supervised** методах (обучение с учителем):

* У нас есть пары `(признаки, класс)`
* Классификатор обучается на них — подбирает подходящую функцию отображения признаков в множество классов
* После этого можно применять полученную модель для предсказаний на новых данных

Признаки могут быть разные:
* средняя длина слова
* минимальная длина слова
* максимальная длина слова
* …

Для определения языка часто используют не слова, а последовательности символов (символьные энграммы / character ngrams). Например, 3-граммы:
```
сим, имв, мво, вол, оло, лов
```

Из обучающих данных соберем словарь символьных энграмм $V$. 
Тогда каждый текст сможем представить в виде вектора длины $|V|$, где каждый признак показывает, присутствует ли соответствующая энграмма в тексте. Если эти значения будут просто $0$/$1$, то не учитывается "важность" последовательности для языка.

Будем использовать $tf \cdot idf$:

$tf \cdot idf (n, d) = \frac{count(n_d)}{\sum_{w \in d}count(w)} \cdot log\frac{|D|}{|\{d \in D | n \in d\}|}$

где $n$ - энграмма, $d$ - документ, а $D$ - весь корпус (на данном языке)

In [0]:
from sklearn import feature_extraction

vectorizer = feature_extraction.text.TfidfVectorizer(ngram_range=(1, 5), analyzer='char')
vectorizer.fit(wiki_texts['ru'])
for item in vectorizer.get_feature_names()[:100]:
    print(item)

In [0]:
print(vectorizer.transform(wiki_texts['ru'])[0])  # первый документ в векторном представлении

In [0]:
#!pip install matplotlib==3.0.3
#!pip install seaborn

In [0]:
from sklearn import pipeline
from sklearn import naive_bayes
import numpy as np

%pylab inline
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
clf = pipeline.Pipeline([
    ('vctr', feature_extraction.text.TfidfVectorizer(ngram_range=(1, 2), analyzer='char')),
    ('clf', naive_bayes.MultinomialNB())
])

In [0]:
all_texts = []
lang_indices = []
for lang in wiki_texts:
    all_texts.extend(wiki_texts[lang])
    lang_indices.extend([lang]*len(wiki_texts[lang]))

In [0]:
# Обучаем классификатор
clf.fit(np.array(all_texts), np.array(lang_indices))

In [0]:
# Предсказываем результаты для тех же текстов
clf.predict(all_texts)

In [0]:
# Скачиваем новые тексты из вики и предсказываем еще раз
clf.predict(get_texts_for_lang('be'))

In [0]:
# Поделим корпус на train и test
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(all_texts,
                                                    lang_indices,
                                                    test_size=0.2,
                                                    random_state=0)
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_predicted)

In [0]:
print(y_predicted)
print(y_test)

In [0]:
# Нарисуем confusion matrix и оценим качество
def test_classify(y_test, y_predicted, label_names):
    cm = metrics.confusion_matrix(y_test, y_predicted)
    
    plt.figure(figsize=(10, 10))
    plt.ylim(10.5, -0.5)
    sns.heatmap(cm, annot=True,  fmt='', xticklabels=label_names, yticklabels=label_names)
    plt.title('Confusion Matrix')
    
    print(metrics.classification_report(y_test, y_predicted,
                                        target_names=label_names))

In [0]:
test_classify(y_test, y_predicted, clf.classes_)

In [0]:
# Проверим на случайных отрывках из тестовых текстов
import random

small_texts = []
for text in X_test:
    begin = random.randint(0, len(text) - 10)
    small_texts.append(text[begin:begin+10])
y_predicted_small = clf.predict(small_texts)
test_classify(y_test, y_predicted_small, clf.classes_)